In [1]:
import pandas as pd
from keras.layers import *
from keras.models import *
from keras import regularizers

Using TensorFlow backend.


In [2]:
train=pd.read_csv('D:/Documents/Downloads/used_car_train.csv')
test=pd.read_csv('D:/Documents/Downloads/used_car_test.csv')
target=pd.read_csv('D:/Documents/Downloads/target.csv')

In [9]:

def NN_model(input_dim):
    reg=Sequential()
    reg.add(Dense(units=200,input_shape=(input_dim,),kernel_initializer='glorot_uniform',
                  kernel_regularizer=regularizers.l1(0.01),activation='softplus'))
    reg.add(Dense(units=64,kernel_initializer='glorot_uniform',activation='softplus'))
    reg.add(Dense(units=32,kernel_initializer='glorot_uniform',activation='softplus'))
    reg.add(Dense(units=8, kernel_initializer='glorot_uniform',activation='softplus'))
    reg.add(Dense(units=1))
    return reg

In [10]:
model=NN_model(train.shape[1])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 200)               18400     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                12864     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 9         
Total params: 33,617
Trainable params: 33,617
Non-trainable params: 0
_________________________________________________________________


In [5]:
from keras import optimizers

In [6]:
epoch=1
b_size=1500
opt=optimizers.SGD(learning_rate=0.01,decay=0.01/epoch,momentum=0.9,nesterov=True)
model.compile(loss='mae',optimizer=opt,metrics=['mae'])
history=model.fit(x=train,y=target,validation_split=0.2,batch_size=b_size, epochs=epoch,shuffle = False)



Instructions for updating:
Use tf.cast instead.
Train on 120000 samples, validate on 30000 samples
Epoch 1/1
120000/120000 [==============================] - 4s 30us/step - loss: 6370.1874 - mae: 6206.1743 - val_loss: 6097.6614 - val_mae: 5891.1426


In [13]:
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping,callbacks
from keras.callbacks import LearningRateScheduler
import math 

def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.2
    epochs_drop = 5.0
    lrate = initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

n_split=5
K=KFold(n_splits=n_split)

b_size=1500
max_epochs=1
x=train.values
y=target.values
for fold,(tra_idx,val_idx) in enumerate(K.split(x,y)):
    print('fold:',fold)
    X_train,y_train=x[tra_idx],y[tra_idx]
    X_val,y_val=x[val_idx],y[val_idx]
    model=NN_model(91)
    opt=optimizers.Adam(lr=0.015)
    model.compile(optimizer=opt,loss='mae',metrics=['mae'])
    es = EarlyStopping(monitor='val_score', patience=5, 
                       verbose=2, mode='min',restore_best_weights=True)
    es.set_model(model)
    model.fit(X_train, y_train, batch_size=b_size, epochs=max_epochs, 
              validation_data = [X_val, y_val],
              callbacks=[lrate], shuffle=True, verbose=2)

fold: 0
Train on 120000 samples, validate on 30000 samples
Epoch 1/1
 - 3s - loss: 2034.6034 - mae: 1983.1505 - val_loss: 1111.5808 - val_mae: 1054.2446
fold: 1
Train on 120000 samples, validate on 30000 samples
Epoch 1/1
 - 4s - loss: 1942.7608 - mae: 1900.8298 - val_loss: 1160.0779 - val_mae: 1115.8635
fold: 2
Train on 120000 samples, validate on 30000 samples
Epoch 1/1
 - 4s - loss: 1593.3762 - mae: 1548.8121 - val_loss: 1169.6701 - val_mae: 1123.1195
fold: 3
Train on 120000 samples, validate on 30000 samples
Epoch 1/1
 - 3s - loss: 1534.2492 - mae: 1491.8263 - val_loss: 1061.8437 - val_mae: 1017.3713
fold: 4
Train on 120000 samples, validate on 30000 samples
Epoch 1/1
 - 4s - loss: 1606.1932 - mae: 1557.9359 - val_loss: 1161.3056 - val_mae: 1109.9398
